In [93]:
import nltk
from nltk.tree import Tree
import sys 
import random 

In [94]:
grammar = nltk.CFG.fromstring("""
    S -> DP VP
    DP -> Det NP | NP | DP Conj DP
    NP -> N | PropN | Adj NP | NP PP
    VP -> Vi | Vt  | Vbar DP | Vbar DP PP | Adv VP | VP Adv | VP Conj VP | V AdvP
    Vbar -> Vi | Vt
    AdvP -> Adv P
    PP -> P DP
    Det -> 'a' | 'an' | 'the' | 'every'| 'some' | 'any'
    P -> 'with' | 'in' | 'on' | 'to' | 'without' | 'from'
    Conj -> 'and' | 'or' | 'but'


    N -> 'boy' | 'student' | 'girl' | 'class' | 'book' | 'teacher'
    PropN -> 'john' | 'mary'
    Adj -> 'eager' | 'smart'
    Vi -> 'walks' | 'passed' 
    Vt -> 'sees' | 'teaches'
    Adv -> 'eagerly' | 'well'
    
""")

In [95]:
sentences = [ 'John sees Mary', 'A student walks', 'Some girl sees every boy','Every eager student passed']

In [96]:
# sentence = random.choice(sentences)
sentence = sentences[0]
parser = nltk.parse.BottomUpLeftCornerChartParser(grammar)                                  
tokens = sentence.lower().split()
parse_trees = parser.parse(tokens)
if not parse_trees:
    print("No parse tree found.")
for parse_tree in parse_trees:
    print(parse_tree)

(S
  (DP (NP (PropN john)))
  (VP (Vbar (Vt sees)) (DP (NP (PropN mary)))))


assignment 2

In [97]:
parser = nltk.parse.BottomUpLeftCornerChartParser(grammar)                                
tokens = sentence.lower().split()
parse_trees = parser.parse(tokens)
if not parse_trees:
    print("No parse tree found.")
for parse_tree in parse_trees:
    parse_tree.pretty_print()

       S            
   ____|____         
  |         VP      
  |     ____|____    
  DP   |         DP 
  |    |         |   
  NP  Vbar       NP 
  |    |         |   
PropN  Vt      PropN
  |    |         |   
 john sees      mary



In [98]:
#select the verb for the sentence 'john sees mary' 
print(parse_tree[1,0])
#select the subject for the sentence 'john sees mary'
print(parse_tree[0])
#select the determiner for sentence 'A student walks'
sentence = sentences[1]
tokens = sentence.lower().split()
parse_trees = parser.parse(tokens)
for parse_tree in parse_trees:
    parse_tree.pretty_print()
print(parse_tree[0,0])
print('\n' )

#practice iterating
for subtree in parse_tree.subtrees(): 
    print(subtree)
print('\n' )
for pos in parse_tree.treepositions(): 
    print(pos)
print('\n')
for pos in parse_tree.treepositions(order="leaves"):
    print(pos)
print('\n')
for pos in parse_tree.treepositions(order="postorder"):
  print(pos)
print('\n')

#get parent node 

def get_parent_node(child_tuple): 
    if len(child_tuple) >= 1: 
        parent_node = child_tuple[:-1]
        return parent_node
    else: 
        return None

#get verb tuple 
def get_verb_tuple(tree):
    for node in tree.treepositions(order="leaves"):
        if tree[get_parent_node(node)].label() in ["Vi", "Vt"]:
            return node

verb = get_verb_tuple(parse_tree) #calling the get verb 
print(parse_tree.pretty_print())
print(verb)



# #Define a function that takes a parse tree, and returns the position of the subject DP.
def get_subj_pos(tree):
    VP_pos = (-1,)
    for node in tree.treepositions(order="preorder"):
        if isinstance(tree[node], nltk.Tree): #Check if node has children
            if tree[node].label() == "VP":
                VP_pos = node
            if node[:len(VP_pos)] != VP_pos and tree[node].label() == "DP":
                return node

dp = get_subj_pos(parse_tree)
print(dp)


#Define a function that takes a parse tree, and returns the position of the object DP if one exists, and None otherwise
def get_object_pos(tree):
    VP_pos = (-1,)
    for node in tree.treepositions(order="preorder"):
        if isinstance(tree[node], nltk.Tree): #Check if node has children
            if tree[node].label() == "VP":
                VP_pos = node
            if node[:len(VP_pos)] == VP_pos and tree[node].label() == "DP":
                return node

sentence = sentences[0]
parser = nltk.parse.BottomUpLeftCornerChartParser(grammar)                                  
tokens = sentence.lower().split()
parse_tree = list(parser.parse(tokens))[0]
parse_tree.pretty_print()
obj = get_object_pos(parse_tree)
print(obj)

(Vbar (Vt sees))
(DP (NP (PropN john)))
           S         
      _____|______    
     DP           |  
  ___|_____       |   
 |         NP     VP 
 |         |      |   
Det        N      Vi 
 |         |      |   
 a      student walks

(Det a)


(S (DP (Det a) (NP (N student))) (VP (Vi walks)))
(DP (Det a) (NP (N student)))
(Det a)
(NP (N student))
(N student)
(VP (Vi walks))
(Vi walks)


()
(0,)
(0, 0)
(0, 0, 0)
(0, 1)
(0, 1, 0)
(0, 1, 0, 0)
(1,)
(1, 0)
(1, 0, 0)


(0, 0, 0)
(0, 1, 0, 0)
(1, 0, 0)


(0, 0, 0)
(0, 0)
(0, 1, 0, 0)
(0, 1, 0)
(0, 1)
(0,)
(1, 0, 0)
(1, 0)
(1,)
()


           S         
      _____|______    
     DP           |  
  ___|_____       |   
 |         NP     VP 
 |         |      |   
Det        N      Vi 
 |         |      |   
 a      student walks

None
(1, 0, 0)
(0,)
       S            
   ____|____         
  |         VP      
  |     ____|____    
  DP   |         DP 
  |    |         |   
  NP  Vbar       NP 
  |    |         |   
PropN  Vt    

Assignment 3

In [99]:
translations = { 
    'John sees Mary' : 'SEES(j,m)', 
    'A student walks': 'some x [WALK(x)]', 
    'Some girl sees every boy': 'all y some x [(GIRL(x) ∧ BOY(y)) → SEES(x, y))]',
    'Every eager student passed': ''}

Assignment 4

In [100]:
vocabulary = {
    'P': (
        'with',
        'in',
        'on',
        'to',
        'without',
        'from'
    ),
    'N': (
        'boy',
        'student',
        'girl',
        'class',
        'book',
        'teacher'
    ),
    'PropN': (
        'john',
        'mary'
    ),
    'Vi': (
        'walks',
        'passed'
    ),
    'Vt': (
        'sees',
        'teaches'
    ),
    'Adj': (
        'eager',
        'smart'
    ),
    'Adv': (
        'eagerly',
        'well'
    ),
    'Det': (
        'a',
        'an',
        'the',
        'every',
        'some',
        'any'
    ),
    'Conj': (
        'and',
        'or',
        'but'
    )
}



Assignment 5

In [101]:
# this class would create an object storing both the representation of an element, and its typing information
class Formalization:
    def __init__(self, formula, type_hint, type_=None):
        self.formula = formula
        self.type = type_hint
        self.t = type_
        if type(type_hint) == tuple:
            self.selected = type_hint[0]
            self.returned = type_hint[1]
        else:
            self.selected = None

    def application(self, argument):
        if argument.type == self.selected:
            resulting_formula = self.formula(argument.formula)
            return Formalization(resulting_formula, self.returned, self.t)


# this dictionary contains functions to generate lexical entries for different parts of speech.
formalizations = {
    'PropN': lambda name: Formalization(
        name,
        'e', 'PropN'
    ),
    'N': lambda noun: Formalization(
        lambda x: f'{noun.upper()}({x})',
        ('e', 't'), 'N'
    ),
    'Vi': lambda verb: Formalization(
        lambda x: f'{verb.upper()}({x})',
        ('e', 't'), 'Vi'
    ),
    'Vt': lambda verb: Formalization(
        lambda y: lambda x: f'{verb.upper()}({x}, {y})',
        ('e', ('e', 't')), 'Vt'
    ),
    'Adj': lambda adjective: Formalization(
        lambda P: lambda x: f'{adjective.upper()}({x}) ^ {P(x)}',
        (('e', 't'), ('e', 't')), 'Adj'
    ),
    'Adv': lambda adverb: Formalization(
        lambda P: lambda x: f'{adverb.upper()}({P(x)})',
        (('e', 't'), ('e', 't')), 'Adv'
    ),
    'P': lambda preposition: Formalization(
        lambda x: lambda y: f'{preposition.upper()}({x}, {y})',
        ('e', ('e', 't')), 'P'
    ),
    'existential': lambda _: Formalization(
        lambda P: lambda Q: f'Exists x[{P("x")} ^ {Q("x")}]',
        (('e', 't'), (('e', 't'), 't')), 'existential'
    ),
    'universal': lambda _: Formalization(
        lambda P: lambda Q: f'All x[{P("x")} -> {Q("x")}]',
        (('e', 't'), (('e', 't'), 't')), 'universal'
    )
}


def generate_lexicon(vocab, translations):
    lexicon = {}
    for part_of_speech, entries in vocab.items():
        if part_of_speech in translations.keys():
            lexicon.update({part_of_speech: {word: translations[part_of_speech](word) for word in entries}})
        elif type(entries) == dict:
            lexicon[part_of_speech] = {}
            for subtype, words in entries.items():
                lexicon[part_of_speech].update({word: translations[subtype](word) for word in words})
        else:
            lexicon.update({part_of_speech: {word: Formalization(word, 'e') for word in entries}})
    return lexicon


def lexicon_to_terminals(lexicon):
    terminal_rules = ''
    for part_of_speech in lexicon.keys():
        terminals_string = " | ".join(["'" + word + "'" for word in lexicon[part_of_speech].keys()])
        terminal_rules += (part_of_speech + ' -> ' + terminals_string + '\n\t')
    return terminal_rules


extensional_lexicon = generate_lexicon(vocabulary, formalizations)

terminals_entries = lexicon_to_terminals(extensional_lexicon)

extensional_grammar = f"""
    S -> NP VP
    NP -> Det Nom | PropN
    Nom -> N  | Adj Nom | Nom PP
    VP -> Vi | Vt  | Vbar NP | Vbar NP PP | Adv VP | VP Adv | VP Conj VP | V AdvP
    Vbar -> Vi | Vt
    AdvP -> Adv P
    PP -> P NP
    {terminals_entries}
"""

Assignment 6

In [102]:
existentials = ['a', 'some', 'an']
universal = ['every']

def check_quantificational_children(result, node):
    return result[node + (1,)].t in ['existential', 'universal'] or result[node + (0,)].t in ['existential', 'universal']

def translate_to_logic(tree = nltk.Tree):
    result = {}
    prev_was_leave = False
    leaf = ''
    root_cause = None
    for node in tree.treepositions(order='postorder'):
        if isinstance(tree[node], nltk.Tree):
            if len(tree[node]) > 1:
                if tree[node].label() == 'S':
                    if check_quantificational_children(result, node):
                        print("help")
                        result[node] = result[node + (0,)].application(result[node + (1,)])
                    else:
                        print(result[node + (0,)])
                        result[node] = result[node + (1,)].application(result[node + (0,)])
                else:
                    result[node] = result[node + (0,)].application(result[node + (1,)])
                    if tree[node].label() == 'VP' and check_quantificational_children(result, node):
                        return None
            else:
                if prev_was_leave:
                    if leaf in existentials:
                        applier = 'existential'
                    elif leaf in universal:
                        applier = 'universal'
                    else:
                        applier = tree[node].label()
                    result[node] = formalizations[applier](leaf)
                    root_cause = result[node]
                    prev_was_leave = False
                else:
                    result[node] = root_cause
        else:
            prev_was_leave = True
            leaf = tree[node]
    return result
sentence = sentences[1]
parser = nltk.parse.BottomUpLeftCornerChartParser(grammar)                                  
tokens = sentence.lower().split()
parse_tree = list(parser.parse(tokens))[0]
parse_tree.pretty_print()
res = translate_to_logic(parse_tree)
res[()].formula

           S         
      _____|______    
     DP           |  
  ___|_____       |   
 |         NP     VP 
 |         |      |   
Det        N      Vi 
 |         |      |   
 a      student walks

help


'Exists x[STUDENT(x) ^ WALKS(x)]'

In [103]:
res

{(0, 0): <__main__.Formalization at 0x20f81d214d0>,
 (0, 1, 0): <__main__.Formalization at 0x20f83052190>,
 (0, 1): <__main__.Formalization at 0x20f83052190>,
 (0,): <__main__.Formalization at 0x20f8317c590>,
 (1, 0): <__main__.Formalization at 0x20f82ffa510>,
 (1,): <__main__.Formalization at 0x20f82ffa510>,
 (): <__main__.Formalization at 0x20f8308c590>}

In [104]:
[print(r.type, r.formula, k) for k, r in res.items() if r is not None]

(('e', 't'), (('e', 't'), 't')) <function <lambda>.<locals>.<lambda> at 0x0000020F82CA3D80> (0, 0)
('e', 't') <function <lambda>.<locals>.<lambda> at 0x0000020F82CA3A60> (0, 1, 0)
('e', 't') <function <lambda>.<locals>.<lambda> at 0x0000020F82CA3A60> (0, 1)
(('e', 't'), 't') <function <lambda>.<locals>.<lambda>.<locals>.<lambda> at 0x0000020F82CA3C40> (0,)
('e', 't') <function <lambda>.<locals>.<lambda> at 0x0000020F82CA3CE0> (1, 0)
('e', 't') <function <lambda>.<locals>.<lambda> at 0x0000020F82CA3CE0> (1,)
t Exists x[STUDENT(x) ^ WALKS(x)] ()


[None, None, None, None, None, None, None]

Assignment 7

In [106]:
def quantifier_raising(tree, nb, position):
    tree_ = tree.copy(deep=True)
    for i in range(nb-1):
        newposition = position + (1,)
        position = newposition
        # print(f'the position is {position}')
    
    qp = tree_[position]
    tree_[position] = f't{nb}'
    print('after trace added \n')
    tree_.pretty_print()

    tree_ = nltk.Tree('S',[qp, tree_])
    tree_[0].set_label(f'DP{nb}')
    return tree_


def raiser(tree=nltk.Tree):
    tree2 = tree.copy(deep=True)
    quant_dict = {}
    for node in tree2.treepositions(order='postorder'):
        if (tree2[node] in existentials) or (tree2[node] in universal):
            quant_dict[node[:-2]] = tree2[node[:-2]]
    print(quant_dict)
    nb = 0
    for key in quant_dict.keys():
        print(quant_dict[key])
        nb += 1
        tree2 = quantifier_raising(tree2, nb, key)
        tree2.pretty_print()
    return tree2

raiser(parse_tree).pretty_print()

{(0,): Tree('DP', [Tree('Det', ['a']), Tree('NP', [Tree('N', ['student'])])])}
(DP (Det a) (NP (N student)))
after trace added 

     S       
  ___|____    
 |        VP 
 |        |   
 |        Vi 
 |        |   
 t1     walks

                 S               
      ___________|_______         
    DP1                  S       
  ___|_____           ___|____    
 |         NP        |        VP 
 |         |         |        |   
Det        N         |        Vi 
 |         |         |        |   
 a      student      t1     walks

                 S               
      ___________|_______         
    DP1                  S       
  ___|_____           ___|____    
 |         NP        |        VP 
 |         |         |        |   
Det        N         |        Vi 
 |         |         |        |   
 a      student      t1     walks

